In [15]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings
import gc

In [16]:
model1=pd.read_hdf('stacking_lgb8568.h5', key='data').reset_index(drop=True)
model1.columns=['lgb_nonn']
model2=pd.read_hdf('stacking849.h5', key='data').reset_index(drop=True)
model2.columns=['nn_fea']
model3=pd.read_hdf('stacking86148.h5', key='data').reset_index(drop=True)
model4=pd.read_hdf('stacking_nn_no_fea_cv7324.h5', key='data').reset_index(drop=True)
model5=pd.read_hdf('stacking_nn_char_cv7320.h5', key='data').reset_index(drop=True)
model6=pd.read_hdf('stacking_nn_LSTUR_cv68.h5', key='data').reset_index(drop=True)
model7=pd.read_hdf('stackingesim.h5', key='data').reset_index(drop=True)
model8=pd.read_hdf('stacking_lgb2.h5', key='data').reset_index(drop=True)

feature_topic_w2vdis=pd.read_hdf('stacking_nn_LSTUR_cv867.h5', key='data')

In [17]:
model3

,nn_feature
0,0.003649
1,0.014941
2,0.018625
3,0.001115
4,0.011770
5,0.071229
6,0.134450
7,0.219796
8,0.003629
9,0.149206


In [19]:
model8

,0
0,5.379168e-03
1,3.280587e-02
2,1.813744e-02
3,1.991170e-03
4,2.912545e-02
5,1.774020e-01
6,8.765377e-02
7,1.421889e-01
8,1.231442e-03
9,1.325881e-01


In [4]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
target=train['target']

In [5]:
train=pd.concat([train,
                 model1.iloc[0:train.shape[0],:],
                 model2.iloc[0:train.shape[0],:],
                 model3.iloc[0:train.shape[0],:],
                 model4.iloc[0:train.shape[0],:],
                 model5.iloc[0:train.shape[0],:],
                 model6.iloc[0:train.shape[0],:], 
                 model7.iloc[0:train.shape[0],:],
                 feature_topic_w2vdis.iloc[0:train.shape[0],:],
                 model8.iloc[0:train.shape[0],:],
                ],axis=1)
test=pd.concat([test,
                model1.iloc[train.shape[0]:,:].reset_index(drop=True),
                model2.iloc[train.shape[0]:,:].reset_index(drop=True),
                model3.iloc[train.shape[0]:,:].reset_index(drop=True),
                model4.iloc[train.shape[0]:,:].reset_index(drop=True),
                model5.iloc[train.shape[0]:,:].reset_index(drop=True),
                model6.iloc[train.shape[0]:,:].reset_index(drop=True),
                model7.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_topic_w2vdis.iloc[train.shape[0]:,:].reset_index(drop=True),
                model8.iloc[train.shape[0]:,:].reset_index(drop=True),
               ],axis=1)

In [6]:
test = test.drop(['uid','qid','time'], axis=1)
train = train.drop(['target', 'uid','qid','time'], axis=1)

In [7]:
train.head()

,lgb_nonn,nn_fea,nn_feature,nn_no_fea,nn_char,nn_LSTUR,esim_feature,nn_LSTURPro,0
0,0.001771,0.001096,0.003649,0.157505,0.074546,0.120644,0.000879,0.000986,0.005379
1,0.028877,0.019841,0.014941,0.243975,0.122339,0.175346,0.024112,0.050738,0.032806
2,0.017263,0.013798,0.018625,0.089800,0.036616,0.054157,0.063057,0.053058,0.018137
3,0.003853,0.001041,0.001115,0.000979,0.029673,0.113823,0.000303,0.006831,0.001991
4,0.022466,0.009935,0.011770,0.192788,0.175256,0.232572,0.019896,0.004325,0.029125


In [8]:
param = {'num_leaves': 30,
         'objective':'binary',
         'learning_rate': 0.1,         
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1,
         "num_threads" : -1,
         }

In [9]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
feature_importance_df = pd.DataFrame()
predictions=np.zeros([len(test),1])
stack_train = np.zeros((len(train),1))
stack_test = np.zeros((len(test),1))

In [10]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
#for trn_idx, val_idx in folds.split(train.values, groups=uid):

    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx],
                           label=target.iloc[trn_idx],
                           )
    val_data = lgb.Dataset(train.iloc[val_idx],
                           label=target.iloc[val_idx],
                           )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=10,
                    early_stopping_rounds=200)
    #stack_train[val_idx, :] =clf.predict(train.iloc[val_idx], num_iteration=clf.best_iteration).reshape(-1,1)
    stack_test += clf.predict(test, num_iteration=clf.best_iteration).reshape(-1,1)
    print(stack_test.mean())

fold n°0
Training until validation scores don't improve for 200 rounds
[10]	training's auc: 0.8934	valid_1's auc: 0.892921
[20]	training's auc: 0.893768	valid_1's auc: 0.893278
[30]	training's auc: 0.893944	valid_1's auc: 0.893469
[40]	training's auc: 0.894062	valid_1's auc: 0.893581
[50]	training's auc: 0.89415	valid_1's auc: 0.893662
[60]	training's auc: 0.894221	valid_1's auc: 0.893722
[70]	training's auc: 0.894278	valid_1's auc: 0.893759
[80]	training's auc: 0.894316	valid_1's auc: 0.893781
[90]	training's auc: 0.894351	valid_1's auc: 0.893781
[100]	training's auc: 0.894348	valid_1's auc: 0.89374
[110]	training's auc: 0.894359	valid_1's auc: 0.893725
[120]	training's auc: 0.894367	valid_1's auc: 0.893722
[130]	training's auc: 0.894357	valid_1's auc: 0.8937
[140]	training's auc: 0.89434	valid_1's auc: 0.893665
[150]	training's auc: 0.892992	valid_1's auc: 0.892187
[160]	training's auc: 0.894201	valid_1's auc: 0.893517
[170]	training's auc: 0.894304	valid_1's auc: 0.893632
[180]	trai

[210]	training's auc: 0.894384	valid_1's auc: 0.893649
[220]	training's auc: 0.894015	valid_1's auc: 0.893271
[230]	training's auc: 0.89432	valid_1's auc: 0.893599
[240]	training's auc: 0.894352	valid_1's auc: 0.893634
[250]	training's auc: 0.8943	valid_1's auc: 0.89359
[260]	training's auc: 0.894375	valid_1's auc: 0.893653
[270]	training's auc: 0.894318	valid_1's auc: 0.893606
[280]	training's auc: 0.894335	valid_1's auc: 0.893635
[290]	training's auc: 0.894318	valid_1's auc: 0.893623
Early stopping, best iteration is:
[91]	training's auc: 0.894371	valid_1's auc: 0.893773
1.0304924619901406


In [13]:
stack_test.mean()

1.0304924619901406

In [14]:
test_id=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
test_id['target']=stack_test/5
test_id.to_csv('result_zf1216.txt',index=False, sep='\t',header=None)

In [12]:
stack_test.mean()

0.6099031622110261

In [13]:
clf.predict(test, num_iteration=clf.best_iteration).reshape(-1,1).mean()

0.2064356970637019